In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow import keras
from tensorflow.keras import layers
import math, os, re, warnings, random
from tensorflow.keras import optimizers, applications, Sequential, losses, metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.preprocessing import LabelEncoder


In [ ]:

from numpy.random import seed
seed(1)
#from tensorflow import set_random_seed
#set_random_seed(2)

In [ ]:
train = pd.read_csv('../input/instadeep-enzyme-classification-challenge/Train (4).csv')
test = pd.read_csv('../input/instadeep-enzyme-classification-challenge/Test (3).csv')

In [ ]:
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']#,'B',X','U','Z']
#Remove redundant samples
train=train.drop_duplicates(subset=['SEQUENCE', 'LABEL'], keep='first')

def get_code_freq(df, data_name):
  
  df = df.apply(lambda x: " ".join(x))
  
  codes = []
  for i in df: # concatination of all codes
    codes.extend(i)

  codes_dict= Counter(codes)
  codes_dict.pop(' ') # removing white space
  print(f'Codes: {data_name}')
  print(f'Total unique codes: {len(codes_dict.keys())}')
  df = pd.DataFrame({'Code': list(codes_dict.keys()), 'Freq': list(codes_dict.values())})
  return df.sort_values('Freq', ascending=False).reset_index()[['Code', 'Freq']]

def create_dict(codes):
  char_dict = {}
  for index, val in enumerate(codes):
    char_dict[val] = index+1
  return char_dict
char_dict = create_dict(codes)

print(char_dict)
print("Dict Length:", len(char_dict))





#apply label encoding to classes
le = LabelEncoder()
train.LABEL = le.fit_transform(train['LABEL'])

In [ ]:
df = df.drop(['SEQUENCE_ID','CREATURE'], axis =1)

In [ ]:

SampleSubmission =pd.read_csv('../input/instadeep-enzyme-classification-challenge/SampleSubmission (3).csv')

In [ ]:

with open('../input/predslstmcnn/pred_CNN_600_4096_32_871.npy', 'rb') as f:
    predCNN = np.load(f)

In [ ]:
with open('../input/predslstmcnn/pred_CNN_600_LSTM_846.npy', 'rb') as f:
    predLSTM = np.load(f)

In [ ]:
#predLSTM.shape

In [ ]:
predCNN55predLSTM45 = 0.55*predCNN + 0.45*predLSTM

In [ ]:
def trans (pred):
    pred= np.argmax(pred, axis=-1)
    pred = le.inverse_transform(pred)
    
    return pred

In [ ]:
predCNN55predLSTM45 = trans(predCNN55predLSTM45)
predCNN55predLSTM45

In [ ]:
predCNN40predLSTM60 = 0.4*predCNN + 0.6*predLSTM

In [ ]:
predCNN40predLSTM60 = trans(predCNN40predLSTM60)
predCNN40predLSTM60

In [ ]:
ss1 = pd.DataFrame({
    'SEQUENCE_ID':SampleSubmission['SEQUENCE_ID'],
    'LABEL':[p for p in  predCNN55predLSTM45]
})
ss1.head()


In [ ]:
ss2 = pd.DataFrame({
    'SEQUENCE_ID':SampleSubmission['SEQUENCE_ID'],
    'LABEL':[p for p in predCNN40predLSTM60]
})
ss2.head()

In [ ]:
ss1.to_csv('CNN55_LSTM_45.csv', index=False)
ss2.to_csv('CNN40_LSTM_60.csv', index=False)